In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.client import device_lib
print(tf.__version__)
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()


1.2.0


[]

# Task 3

In [2]:
heart_data = pd.read_csv("../../data/heart.csv")
print heart_data.info()
print [name for name in heart_data.columns]
print heart_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 10 columns):
sbp          462 non-null int64
tobacco      462 non-null float64
ldl          462 non-null float64
adiposity    462 non-null float64
famhist      462 non-null object
typea        462 non-null int64
obesity      462 non-null float64
alcohol      462 non-null float64
age          462 non-null int64
chd          462 non-null int64
dtypes: float64(5), int64(4), object(1)
memory usage: 36.2+ KB
None
['sbp', 'tobacco', 'ldl', 'adiposity', 'famhist', 'typea', 'obesity', 'alcohol', 'age', 'chd']
   sbp  tobacco   ldl  adiposity  famhist  typea  obesity  alcohol  age  chd
0  160    12.00  5.73      23.11  Present     49    25.30    97.20   52    1
1  144     0.01  4.41      28.61   Absent     55    28.87     2.06   63    1
2  118     0.08  3.48      32.28  Present     52    29.14     3.81   46    0
3  170     7.50  6.41      38.03  Present     51    31.99    24.26   58    1
4  134    13.60 

# Logistic Regression on heart data

In [12]:
BATCH_SIZE = 3
DATA_PATH = '../../data/heart.csv'
N_FEATURES = 9

EPOCHS = 1000
LEARNING_RATE=0.5

### data1 

In [8]:
# Step 1: get data

def batch_generator(filenames):
    """ filenames is the list of files you want to read from. 
    In this case, it contains only heart.csv
    """
    filename_queue = tf.train.string_input_producer(filenames)
    reader = tf.TextLineReader(skip_header_lines=1) # skip the first line in the file
    _, value = reader.read(filename_queue)

    # record_defaults are the default values in case some of our columns are empty
    # This is also to tell tensorflow the format of our data (the type of the decode result)
    # for this dataset, out of 9 feature columns, 
    # 8 of them are floats (some are integers, but to make our features homogenous, 
    # we consider them floats), and 1 is string (at position 5)
    # the last column corresponds to the lable is an integer

    record_defaults = [[1.0] for _ in range(N_FEATURES)]
    record_defaults[4] = ['']
    record_defaults.append([1])

    # read in the 10 rows of data
    content = tf.decode_csv(value, record_defaults=record_defaults) 

    # convert the 5th column (present/absent) to the binary value 0 and 1
    condition = tf.equal(content[4], tf.constant('Present'))
    content[4] = tf.cond(condition, lambda: tf.constant(1.0), lambda: tf.constant(0.0))

    # pack all 9 features into a tensor
    features = tf.stack(content[:N_FEATURES])

    # assign the last column to label
    label = content[-1]

    # minimum number elements in the queue after a dequeue, used to ensure 
    # that the samples are sufficiently mixed
    # I think 10 times the BATCH_SIZE is sufficient
    min_after_dequeue = 10 * BATCH_SIZE

    # the maximum number of elements in the queue
    capacity = 20 * BATCH_SIZE

    # shuffle the data to generate BATCH_SIZE sample pairs
    data_batch, label_batch = tf.train.shuffle_batch([features, label], batch_size=BATCH_SIZE, 
                                        capacity=capacity, min_after_dequeue=min_after_dequeue)

    return data_batch, label_batch

def generate_batches(data_batch, label_batch):
    with tf.Session() as sess:
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        for _ in range(10): # generate 10 batches
            features, labels = sess.run([data_batch, label_batch])
            print features
        coord.request_stop()
        coord.join(threads)

data1_feature_batch, data1_label_batch = batch_generator([DATA_PATH])
# generate_batches(data_batch, label_batch)

### data2

In [9]:
def read_row(csv_row):
    record_defaults = [[0], [0.0], [0.0], [0.0], [""], [0], [0.0], [0.0], [0], [0]]
    row = tf.decode_csv(csv_row, record_defaults=record_defaults)
    row[4] = tf.cond(tf.equal(row[4], "Present"), lambda: 1, lambda: 0)
    return row[:-1], row[-1]

def input_pipeline(filenames, batch_size):
    # Define a `tf.contrib.data.Dataset` for iterating over one epoch of the data.
    dataset = (tf.contrib.data.TextLineDataset(filenames)
               .skip(1)
               .map(lambda line: read_row(line))
               .shuffle(buffer_size=10)  # Equivalent to min_after_dequeue=10.
               .batch(batch_size))

    # Return an *initializable* iterator over the dataset, which will allow us to
    # re-initialize it at the beginning of each epoch.
    return dataset.make_initializable_iterator()

data_iterator = input_pipeline([DATA_PATH], BATCH_SIZE)
data2_feature_batch, data2_label_batch = data_iterator.get_next()


In [10]:

# Step 2: create placeholders for input X (Features) and label Y (binary result)
X = tf.placeholder(tf.float32, shape=[None, 9], name="X")
Y = tf.placeholder(tf.float32, shape=[None], name="Y")

# Step 3: create weight and bias, initialized to 0
w = tf.Variable(tf.truncated_normal([9, 1]), name="weights")
b = tf.Variable(tf.zeros([1,1]), name="bias")

# Step 4: logistic multinomial regression / softmax
score = tf.matmul(X, w) + b

# Step 5: define loss function
loss = tf.nn.softmax_cross_entropy_with_logits(logits=tf.transpose(score), labels=Y, name="loss")

# Step 6: using gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE).minimize(loss)

# Step 7: Prediction
Y_predicted = tf.nn.softmax(tf.matmul(X, w) + b)
correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(Y_predicted,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(X)
print(Y)
print(w)
print(b)

print(score)
print(loss)
print(optimizer)
print(Y_predicted)
print(correct_prediction)
print(accuracy)

Tensor("X_1:0", shape=(?, 9), dtype=float32)
Tensor("Y_1:0", shape=(?,), dtype=float32)
<tf.Variable 'weights_1:0' shape=(9, 1) dtype=float32_ref>
<tf.Variable 'bias_1:0' shape=(1, 1) dtype=float32_ref>
Tensor("add_2:0", shape=(?, 1), dtype=float32)
Tensor("Reshape_5:0", shape=(1,), dtype=float32)
name: "GradientDescent_1"
op: "NoOp"
input: "^GradientDescent_1/update_weights_1/ApplyGradientDescent"
input: "^GradientDescent_1/update_bias_1/ApplyGradientDescent"

Tensor("Softmax_1:0", shape=(?, 1), dtype=float32)
Tensor("Equal_2:0", shape=(?,), dtype=bool)
Tensor("Mean_1:0", shape=(), dtype=float32)


### data1 run

In [11]:

with tf.Session() as sess:
    # Step 7: initialize the necessary variables, in this case, w and b
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # Step 8: train the model
    for i in tqdm(range(EPOCHS)): # run epochs
        x_batch, y_batch = sess.run([data1_feature_batch, data1_label_batch])
        # Session runs train_op to minimize loss
        feed_dict={X: x_batch, Y:y_batch}
        
        sess.run(optimizer, feed_dict=feed_dict)
        
    # Step 9: output the values of w and b
    w_value, b_value = sess.run([w, b])

    # Step 10: predict
    # print(sess.run(accuracy, feed_dict={X: mnist.validation.images, Y:mnist.validation.labels}))

    coord.request_stop()
    coord.join(threads)


100%|██████████| 10000/10000 [00:05<00:00, 1747.45it/s]


### data2 run

In [14]:
with tf.Session() as sess:
    # Step 7: initialize the necessary variables, in this case, w and b
    tf.global_variables_initializer().run()
    sess.run(data_iterator.initializer)
    
    # Step 8: train the model
    for i in tqdm(range(EPOCHS)): # run epochs
        x_batch, y_batch = sess.run([data2_feature_batch, data2_label_batch])
        print(np.shape(data2_feature_batch))
        # Session runs train_op to minimize loss
        feed_dict={X: np.transpose(x_batch), Y:y_batch}
        sess.run(optimizer, feed_dict=feed_dict)
        
    # Step 9: output the values of w and b
    w_value, b_value = sess.run([w, b])

    # Step 10: predict
    # print(sess.run(accuracy, feed_dict={X: mnist.validation.images, Y:mnist.validation.labels}))


 10%|█         | 100/1000 [00:00<00:01, 473.20it/s]

(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)


(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)
(9,)


OutOfRangeError: End of sequence
	 [[Node: IteratorGetNext_1 = IteratorGetNext[output_shapes=[[-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]], output_types=[DT_INT32, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT32, DT_INT32, DT_FLOAT, DT_FLOAT, DT_INT32, DT_INT32], _device="/job:localhost/replica:0/task:0/cpu:0"](Iterator_1)]]

Caused by op u'IteratorGetNext_1', defined at:
  File "/home/bicepjai/Programs/anaconda2/envs/deepl/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/bicepjai/Programs/anaconda2/envs/deepl/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/bicepjai/Programs/anaconda2/envs/deepl/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/bicepjai/Programs/anaconda2/envs/deepl/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/bicepjai/